In [ ]:
#from google.colab import drive
#drive.mount('/content/gdrive')

In [ ]:
#from google.colab import files
#files.upload()

In [ ]:
#!ls -lha kaggle.json

In [5]:
#!pip install -q kaggle

In [6]:
#!mkdir -p ~/.kaggle
#!cp kaggle.json ~/.kaggle/

In [7]:
#!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
#!pwd

In [9]:
#!kaggle datasets list

ref                                                                 title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
------------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
rabieelkharoua/students-performance-dataset                         📚 Students Performance Dataset 📚                     66KB  2024-06-12 23:09:20          12645        266  1.0              
nelgiriyewithana/most-streamed-spotify-songs-2024                   Most Streamed Spotify Songs 2024                    496KB  2024-06-15 18:50:51           9704        207  1.0              
fahmidachowdhury/e-commerce-sales-analysis                          📈 E-Commerce Sales Analysis                          35KB  2024-07-04 20:02:23            935         26  0.9411765        
ihelon/coffee-sales                     

In [ ]:
#!kaggle datasets download -d mohamedbakhet/amazon-books-reviews

In [ ]:
#!unzip amazon-books-reviews.zip

## **Data Ingestion Techniques**

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### **Pandas:**

In [13]:
df_ratings = pd.read_csv('Books_rating.csv')
df_books = pd.read_csv('books_data.csv')

> Pandas took almost ~1 minute to load the data.

In [19]:
# Deleting the dataframes

lst = [df_ratings, df_books]
del lst
del df_ratings, df_books